In [1]:
# import packages

import dask_awkward as dak
import numpy as np
from coffea.dataset_tools import preprocess
from distributed import Client
from matplotlib import pyplot as plt

import egamma_tnp
from egamma_tnp import ElectronTagNProbeFromNanoAOD, ElectronTagNProbeFromNTuples
from egamma_tnp.plot import plot_efficiency

In [2]:
# start client
client = Client()

In [3]:
# define datasets

fileset = {
    "samples": {
        "files": {
            "../root_files/Data_Run2023C_EGamma0_NANOAOD_22Sep2023_v1-v1_001.root": "Events",
        }
    }
}

print(fileset)

fileset_available, fileset_updates = preprocess(fileset, skip_bad_files=True, step_size=500_000)

{'samples': {'files': {'../root_files/Data_Run2023C_EGamma0_NANOAOD_22Sep2023_v1-v1_001.root': 'Events'}}}


In [4]:
# initialize tag and probe class

tnp = ElectronTagNProbeFromNanoAOD(
    fileset_available,
    filters=["HLT_Ele30_WPTight_Gsf"],
    filterbit=[1],
    cutbased_id = "cutBased >= 4",
    tags_pt_cut=30,
    use_sc_eta=True,
    tags_abseta_cut=2.50,
    probes_pt_cut=27,
    trigger_pt=[30],
)

egamma_tnp.binning.set("MET_pt_bins", np.linspace(0, 200, 21).tolist())
egamma_tnp.binning.set("luminosityBlock_bins", np.linspace(0, 1000, 11).tolist())
egamma_tnp.binning.set("tag_Ele_pt_bins", egamma_tnp.binning.get("pt_bins"))
egamma_tnp.binning.set("tag_Ele_eta_bins", egamma_tnp.binning.get("eta_bins"))
egamma_tnp.binning.set("el_pt_bins", egamma_tnp.binning.get("pt_bins"))
egamma_tnp.binning.set("el_eta_bins", egamma_tnp.binning.get("eta_bins"))
egamma_tnp.binning.set("Electron_pt_bins", egamma_tnp.binning.get("pt_bins"))

In [5]:
dak.necessary_columns(tnp.get_tnp_arrays(cut_and_count=False, vars=["Electron_pt", "tag_Ele_eta", "el_pt", "el_eta", "MET_pt", "luminosityBlock"]))

/usr/local/lib/python3.11/site-packages/coffea/nanoevents/methods/candidate.py:11: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


{'from-uproot-40e269e610ed53611be219d880204c9f': frozenset({'Electron_charge',
            'Electron_cutBased',
            'Electron_deltaEtaSC',
            'Electron_eta',
            'Electron_mass',
            'Electron_phi',
            'Electron_pt',
            'HLT_Ele30_WPTight_Gsf',
            'MET_pt',
            'TrigObj_eta',
            'TrigObj_filterBits',
            'TrigObj_id',
            'TrigObj_phi',
            'TrigObj_pt',
            'luminosityBlock',
            'nElectron',
            'nTrigObj'})}

In [6]:
dak.necessary_columns(
    tnp.get_nd_tnp_histograms("HLT_Ele30_WPTight_Gsf", cut_and_count=True, vars=["tag_Ele_pt", "el_pt", "el_eta", "MET_pt", "luminosityBlock"])
)

{'from-uproot-40e269e610ed53611be219d880204c9f': frozenset({'Electron_charge',
            'Electron_cutBased',
            'Electron_deltaEtaSC',
            'Electron_eta',
            'Electron_mass',
            'Electron_phi',
            'Electron_pt',
            'HLT_Ele30_WPTight_Gsf',
            'MET_pt',
            'TrigObj_eta',
            'TrigObj_filterBits',
            'TrigObj_id',
            'TrigObj_phi',
            'TrigObj_pt',
            'luminosityBlock',
            'nElectron',
            'nTrigObj'})}

In [7]:
# cut_and_count=True
#out = tnp.get_tnp_arrays(cut_and_count=True, vars=["Electron_pt", "tag_Ele_eta", "el_pt", "el_eta", "MET_pt", "luminosityBlock"], compute=True)
# cut_and_count=False
out = tnp.get_tnp_arrays(cut_and_count=False, vars=["Electron_pt", "tag_Ele_pt", "tag_Ele_eta", "el_pt", "el_eta", "MET_pt", "luminosityBlock"], compute=True)

/usr/local/lib/python3.11/site-packages/coffea/nanoevents/methods/candidate.py:11: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [8]:
out["samples"]

<Array [{Electron_pt: [...], ...}, ..., {...}] type='7450 * {Electron_pt: v...'>

In [9]:
out["samples"].pair_mass

<Array [[92.1, 92.1], [95.8, ...], ..., [89, 89]] type='7450 * var * float32'>

In [10]:
out["samples"].el_pt

<Array [[38.2, 46.5], [...], ..., [37.7, 54.9]] type='7450 * var * float32[...'>

In [11]:
out = tnp.get_1d_pt_eta_phi_tnp_histograms(
    "HLT_Ele30_WPTight_Gsf",
    cut_and_count=True,
    vars=["el_pt", "el_eta", "el_phi"],
    eta_regions_pt={
        "barrel": [0.0, 1.4442],
        "endcap_loweta": [1.566, 2.0],
        "endcap_higheta": [2.0, 2.5],
    },
    plateau_cut=35,
    compute=True,
)

/usr/local/lib/python3.11/site-packages/coffea/nanoevents/methods/candidate.py:11: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [12]:
# cut_and_count=True
#out = tnp.get_nd_tnp_histograms("HLT_Ele30_WPTight_Gsf", cut_and_count=True, vars=["tag_Ele_pt", "el_pt", "el_eta", "MET_pt", "luminosityBlock"], compute=True)
# cut_and_count=False
out = tnp.get_nd_tnp_histograms("HLT_Ele30_WPTight_Gsf", cut_and_count=False, vars=["Electron_pt", "tag_Ele_pt", "tag_Ele_eta", "el_pt", "el_eta", "MET_pt", "luminosityBlock"], compute=True)

MemoryError: std::bad_alloc

In [ ]:
print(out)

In [ ]:
# Tag electron p_T
out["samples"]["passing"].project("tag_Ele_pt").plot()
out["samples"]["failing"].project("tag_Ele_pt").plot()
plt.show()

In [ ]:
# Probe electron p_T
out["samples"]["passing"].project("el_pt").plot()
out["samples"]["failing"].project("el_pt").plot()
plt.show()

In [ ]:
# MET p_T
out["samples"]["passing"].project("MET_pt").plot()
out["samples"]["failing"].project("MET_pt").plot()
plt.show()

In [ ]:
out["samples"].project("pair_mass").plot()
out["samples"].project("pair_mass").plot()
plt.show()

In [ ]:
# efficiency vs. probe p_T
plot_efficiency(out["samples"]["passing"].project("el_pt"), out["samples"]["failing"].project("el_pt"))
plt.xlim(0.0, 100.0)
plt.ylim(-0.1, 1.1)
plt.title("Electron Efficiency vs. probe p_T")
plt.show()

In [ ]:
# efficiency vs. probe eta
plot_efficiency(out["samples"]["passing"].project("el_eta"), out["samples"]["failing"].project("el_eta"))
plt.xlim(-3.0, 3.0)
plt.ylim(-0.1, 1.1)
plt.title("Electron Efficiency vs. probe eta")
plt.show()